<a href="https://colab.research.google.com/github/tjsiledar/Machine-Learning/blob/master/MemN2N.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#libraries to download data and preprocess it.

import re
import tarfile
import numpy as np
from functools import reduce
from keras.utils.data_utils import get_file
from keras.preprocessing.sequence import pad_sequences

In [0]:
'''
Function to create tokens of a sentence

Returns a list of tokens.
'''

def tokenize(sentence):
  return [x.strip() for x in re.split('(\W+)?', sentence) if x.strip()]

In [0]:
'''
Function to create tuples of each episode
'''

def parse_episodes(lines):
  
  data = []
  story = []
  
  for line in lines:
    line = line.decode('utf-8').strip()
    nid, line = line.split(' ', 1)
    nid = int(nid)
    
    if nid==1:
      story = []
    
    if '\t' in line:
      q, a, support_id = line.split('\t')
      q = tokenize(q)
      substory = [x for x in story if x]
      
      #add substory question and answer as a tuple to the data list
      
      data.append((substory,q,a))
      story.append('')
    else:
      sentence = tokenize(line)
      story.append(sentence)
      
  return data

In [0]:
'''
Function to create a list of all episodes from the file
'''

def get_episodes(file):
  data = parse_episodes(file.readlines())
  flatten = lambda data: reduce(lambda x,y: x+y, data)
  data = [(flatten(story), question, answer) for story, question, answer in data]
  return data

In [0]:
'''
Function to vectorize episodes
'''

def vectorize(data, word_idx, story_maxlen, query_maxlen):
  stories, queries, answers = [], [], []
  
  for story, query, answer in data:
    stories.append([word_idx[w] for w in story])
    queries.append([word_idx[w] for w in query])
    y = np.zeros(len(word_idx)+1)
    y[word_idx[answer]]=1
    answers.append(y)
    
  return (pad_sequences(stories, maxlen=story_maxlen), pad_sequences(queries, maxlen=query_maxlen), np.array(answers))

In [0]:
#downloading the dataset

path = get_file('babi-tasks-v1-2.tar.gz', origin='https://s3.amazonaws.com/text-datasets/babi_tasks_1-20_v1-2.tar.gz')
file = tarfile.open(path)

In [0]:
dataset = {
    
    'single_supporting_fact_10k' : 'tasks_1-20_v1-2/en-10k/qa1_single-supporting-fact_{}.txt',
    'two_supporting_fact_10k' : 'tasks_1-20_v1-2/en-10k/qa2_two-supporting-facts_{}.txt',
}

current_dataset = 'single_supporting_fact_10k'

dataset = dataset[current_dataset]

In [72]:
#Extracting train and test datasets

train_set = get_episodes(file.extractfile(dataset.format('train')))
test_set = get_episodes(file.extractfile(dataset.format('test')))

/usr/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [73]:
#checking our sets

print(len(train_set))
print(len(test_set))

train_set[0]

10000
1000


(['Mary',
  'moved',
  'to',
  'the',
  'bathroom',
  '.',
  'John',
  'went',
  'to',
  'the',
  'hallway',
  '.'],
 ['Where', 'is', 'Mary', '?'],
 'bathroom')

In [0]:
#creating a vocabulary from our sentences

vocab = set()

for story, query, answer in train_set + test_set:
  vocab |= set(story + query + [answer])
vocab = sorted(vocab)

In [0]:
#as 0 is reserved for padding

vocab_size = len(vocab) + 1 

In [0]:
#calculating the maximum length of story and query

story_maxlen = max(map(len, (x for x,_,_ in train_set + test_set)))
query_maxlen = max(map(len, (x for _,x,_ in train_set + test_set)))

In [0]:
# creating word to index dictionary

word_idx = dict((c,i+1) for i,c in enumerate(vocab))

In [78]:
print(word_idx)

{'.': 1, '?': 2, 'Daniel': 3, 'John': 4, 'Mary': 5, 'Sandra': 6, 'Where': 7, 'back': 8, 'bathroom': 9, 'bedroom': 10, 'garden': 11, 'hallway': 12, 'is': 13, 'journeyed': 14, 'kitchen': 15, 'moved': 16, 'office': 17, 'the': 18, 'to': 19, 'travelled': 20, 'went': 21}


In [0]:
# vectorizing story, query and answer using vocab

stories_train, queries_train, answers_train = vectorize(train_set, word_idx, story_maxlen, query_maxlen)
stories_test, queries_test, answers_test = vectorize(test_set, word_idx, story_maxlen, query_maxlen)

In [80]:
stories_train[0]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  5, 16, 19, 18,  9,  1,  4, 21, 19, 18, 12,  1],
      dtype=int32)

In [81]:
queries_train[0]

array([ 7, 13,  5,  2], dtype=int32)

In [82]:
answers_train[0,:]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.])

Lets start buliding our End to End model

In [0]:
import IPython
import matplotlib.pyplot as plt
import pandas as pd
 
import keras
from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Permute, dot, add, concatenate
from keras.layers import LSTM, Dense, Dropout, Input, Activation

In [0]:
# number of epochs to run
train_epochs = 100
# Training batch size
batch_size = 32
# Hidden embedding size
embed_size = 50
# number of nodes in LSTM layer
lstm_size = 64
# dropout rate
dropout_rate = 0.30

In [0]:
#placeholders

input_sequence = Input((story_maxlen,))
question = Input((query_maxlen,))

In [0]:
#encoders

input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size, output_dim=embed_size))
input_encoder_m.add(Dropout(dropout_rate))

input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size, output_dim=query_maxlen))
input_encoder_c.add(Dropout(dropout_rate))

question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_size, output_dim=embed_size, input_length=query_maxlen))
question_encoder.add(Dropout(dropout_rate))

In [0]:
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)

In [0]:
# compute match between first input vector and question vector

match = dot([input_encoded_m, question_encoded], axes=-1, normalize=False)
match = Activation('softmax')(match)

In [0]:
# add match to the second input vector

res = add([match, input_encoded_c])
res = Permute((2,1))(res)

In [0]:
# concatenate the response vector with question vector

answer = concatenate([res, question_encoded])

In [0]:
answer = LSTM(lstm_size)(answer)
answer = Dropout(dropout_rate)(answer)
answer = Dense(vocab_size)(answer)
answer = Activation('softmax')(answer)

In [92]:
# building the model

model = Model([input_sequence, question], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

W0728 12:42:39.603384 140667283019648 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0728 12:42:39.632448 140667283019648 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [93]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 68)           0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 4)            0                                            
__________________________________________________________________________________________________
sequential_5 (Sequential)       multiple             1100        input_3[0][0]                    
__________________________________________________________________________________________________
sequential_7 (Sequential)       (None, 4, 50)        1100        input_4[0][0]                    
__________________________________________________________________________________________________
dot_2 (Dot

In [94]:
model.fit([stories_train, queries_train], answers_train, batch_size, train_epochs, validation_data = ([stories_test, queries_test], answers_test))

W0728 12:45:44.216058 140667283019648 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 10000 samples, validate on 1000 samples
Epoch 1/100
10000/10000 [==============================] - 5s 536us/step - loss: 1.8887 - acc: 0.1704 - val_loss: 1.8136 - val_acc: 0.1490
Epoch 2/100
10000/10000 [==============================] - 4s 354us/step - loss: 1.7185 - acc: 0.2393 - val_loss: 1.6468 - val_acc: 0.2930
Epoch 3/100
10000/10000 [==============================] - 4s 354us/step - loss: 1.6470 - acc: 0.2835 - val_loss: 1.5909 - val_acc: 0.3770
Epoch 4/100
10000/10000 [==============================] - 4s 355us/step - loss: 1.5326 - acc: 0.3785 - val_loss: 1.4854 - val_acc: 0.4020
Epoch 5/100
10000/10000 [==============================] - 4s 352us/step - loss: 1.4799 - acc: 0.3985 - val_loss: 1.4270 - val_acc: 0.4210
Epoch 6/100
10000/10000 [==============================] - 4s 354us/step - loss: 1.4109 - acc: 0.4354 - val_loss: 1.3593 - val_acc: 0.4730
Epoch 7/100
10000/10000 [==============================] - 4s 353us/step - loss: 1.3773 - acc: 0.4522 - val_loss: 1.3